In [ ]:
import cv2
import numpy as np
import glob
import os
from natsort import natsorted

image_folder = "/Users/donglinhan/LargeFiles/Humidity_seeking/HUMI011/"
output_folder = "/Users/donglinhan/LargeFiles/Humidity_seeking/HUMI011/processed_images"
os.makedirs(output_folder, exist_ok=True)

# Initialize global variables for manual ROI selection
points = []
roi_points = []

# Callback function for selecting ROI points
def select_points(event, x, y, flags, param):
    global points
    if event == cv2.EVENT_LBUTTONDOWN and len(points) < 4:  # Collect exactly 4 points for one ROI
        points.append((x, y))
        cv2.circle(frame, (x, y), 5, (0, 255, 0), -1)
        if len(points) == 4:
            cv2.polylines(frame, [np.array(points)], True, (255, 0, 0), 2)
        cv2.imshow("Select ROI", frame)

# Load the first image to define the ROI
all_image_files = [os.path.join(image_folder, img) for img in os.listdir(image_folder) if img.endswith('.jpg')]
image_files = natsorted(all_image_files)
frame = cv2.imread(image_files[0])

if frame is not None:
    cv2.imshow("Select ROI", frame)
    cv2.setMouseCallback("Select ROI", select_points)

    # Wait until 4 points are selected
    while len(points) < 4:
        if cv2.waitKey(1) == ord("c"):  # Clear points if 'c' is pressed
            points.clear()
            frame = cv2.imread(image_files[0]).copy()
            cv2.imshow("Select ROI", frame)

    # Save ROI points and close window
    roi_points = points
    cv2.destroyAllWindows()

# Function to crop the ROI based on selected points
def crop_roi(image, points):
    rect = cv2.boundingRect(np.array(points, dtype="int"))  # Get bounding rectangle
    x, y, w, h = rect
    return image[y:y+h, x:x+w]


# Group images and process each group
n = 60  # Number of images per group
image_groups = [image_files[i:i + n] for i in range(0, len(image_files), n)]

for group_idx, group in enumerate(image_groups):
    # Initialize minimum projection array for the ROI
    first_image = cv2.imread(group[0])
    roi_cropped = crop_roi(first_image, roi_points)
    roi_min = np.full_like(roi_cropped, 255, dtype=np.uint8)

    for image_path in group:
        image = cv2.imread(image_path)
        roi_cropped = crop_roi(image, roi_points)
        roi_min = np.minimum(roi_min, roi_cropped)

    # Save the minimum projection as an image
    output_path = os.path.join(output_folder, f"group_{group_idx + 1}_min_projection.png")
    cv2.imwrite(output_path, roi_min)
    print(f"Saved minimum projection for group {group_idx + 1} at {output_path}")


print("Processing completed.")


In [3]:
import cv2
import numpy as np
import glob
import os
from natsort import natsorted

def create_time_max_projection(folder_path, output_path):
    # Get a list of all image files in the folder
    image_files = natsorted([f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.tif'))])
    
    # Initialize a variable to store the maximum projection
    max_projection = None
    
    for idx, file in enumerate(image_files):
        # Read the image as a grayscale image
        image_path = os.path.join(folder_path, file)
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        
        if img is None:
            print(f"Warning: Could not read {file}. Skipping.")
            continue
        
        # Initialize max_projection with the first image
        if max_projection is None:
            max_projection = img.astype(np.uint16)  # Use uint16 to avoid overflow when summing
        
        # Update the maximum projection
        max_projection = np.maximum(max_projection, img)
    
    # Convert back to uint8 for saving
    max_projection = np.clip(max_projection, 0, 255).astype(np.uint8)
    
    # Save the result
    cv2.imwrite(output_path, max_projection)
    print(f"Time-maximum projection saved to: {output_path}")

# Example usage
folder_path = "/Users/donglinhan/LargeFiles/Humidity_seeking/HUMI011"  # Replace with the folder containing images
output_path = "/Users/donglinhan/LargeFiles/Humidity_seeking/HUMI011/000.jpg"  # Replace with desired output path
create_time_max_projection(folder_path, output_path)


Time-maximum projection saved to: /Users/donglinhan/LargeFiles/Humidity_seeking/HUMI011/000.jpg


In [8]:
import cv2
import os
import numpy as np
import pandas as pd
import natsort  # Import the natsort library

# Global variable to store the ROI coordinates
rois = []

# Mouse callback function to select ROIs
def select_roi(event, x, y, flags, param):
    global rois
    if event == cv2.EVENT_LBUTTONDOWN:
        # Record the coordinates of the click
        rois.append((x, y))
        if len(rois) % 2 == 0:  # After every two points, we will define one ROI
            cv2.rectangle(img, rois[-2], rois[-1], (0, 255, 0), 2)
            cv2.imshow("Image", img)

# Function to calculate pixels below threshold
def calculate_pixels_below_threshold(image, roi, threshold):
    x1, y1 = roi[0]
    x2, y2 = roi[1]
    roi_image = image[y1:y2, x1:x2]
    # Apply threshold
    _, thresholded = cv2.threshold(roi_image, threshold, 255, cv2.THRESH_BINARY_INV)
    # Count non-zero pixels (below threshold)
    return np.count_nonzero(thresholded)

# Load images from folder
image_folder = "/Users/donglinhan/LargeFiles/Humidity_seeking/HUMI007"  # Update this to your folder path
images = [f for f in os.listdir(image_folder) if f.endswith('.jpg')]

# Sort files using natural sorting
images = natsort.natsorted(images)

# Set threshold value (you can adjust this based on your image)
threshold_value = 100

# Store results for Excel file
results = []

# Define ROIs on the first image
first_image_path = os.path.join(image_folder, images[0])
img = cv2.imread(first_image_path, cv2.IMREAD_GRAYSCALE)

# Display the first image and define ROIs
cv2.imshow("Image", img)
cv2.setMouseCallback("Image", select_roi)
print("Click on 8 points to define 4 ROIs (Top-Left, Bottom-Right for each ROI).")
print("After selecting 8 points, press any key to proceed.")
cv2.waitKey(0)
cv2.destroyAllWindows()

# Ensure 8 points were selected for 4 ROIs
if len(rois) == 8:
    # Process all images in the folder
    for image_name in images:
        image_path = os.path.join(image_folder, image_name)
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        
        roi_pixels = []
        for i in range(0, len(rois), 2):
            # Calculate pixels below threshold for each pair of points (ROI)
            pixel_count = calculate_pixels_below_threshold(img, rois[i:i+2], threshold_value)
            roi_pixels.append(pixel_count)

        # Append results for the current image
        results.append([image_name] + roi_pixels)

    # Save results to Excel
    columns = ['Image Name', 'ROI 1 Pixels Below Threshold', 'ROI 2 Pixels Below Threshold', 'ROI 3 Pixels Below Threshold', 'ROI 4 Pixels Below Threshold']
    df = pd.DataFrame(results, columns=columns)
    columns_to_adjust = df.columns[1:5]
    df[columns_to_adjust] = df[columns_to_adjust].apply(lambda col: col - col.iloc[0])
    df['Fed_surplus'] = df['ROI 2 Pixels Below Threshold'] - df['ROI 1 Pixels Below Threshold']
    df['Unfed_surplus'] = df['ROI 4 Pixels Below Threshold'] - df['ROI 3 Pixels Below Threshold']
    excel_path = "/Users/donglinhan/LargeFiles/Humidity_seeking/HUMI007_main.xlsx"
    df.to_excel(excel_path, index=False)

    print("Results saved to roi_pixel_counts.xlsx")
else:
    print("You must select exactly 8 points to define 4 ROIs.")


Click on 8 points to define 4 ROIs (Top-Left, Bottom-Right for each ROI).
After selecting 8 points, press any key to proceed.
Results saved to roi_pixel_counts.xlsx


In [1]:
import pandas as pd
from scipy.stats import ttest_ind

file_path = "/Users/donglinhan/Desktop/EHyper2025/Humidity seeking/CYC.xlsx"
df = pd.read_excel(file_path, engine='openpyxl')


# Prepare lists to store the t-test results
t_statistics = []
p_values = []

# Perform a t-test for each row (time point)
for index, row in df.iterrows():
    condition1 = row.iloc[1:5].values  # Columns 0, 1, 2
    condition2 = row.iloc[5:8].values  
    
    # Perform independent two-sample t-test
    t_stat, p_value = ttest_ind(condition1, condition2, equal_var=False)  # Welch's t-test for unequal variance
    
    # Save the results
    t_statistics.append(t_stat)
    p_values.append(p_value)

# Add results to the DataFrame
df['t_stat'] = t_statistics
df['p_value'] = p_values

excel_path = "/Users/donglinhan/Desktop/EHyper2025/Humidity seeking/CYC_stats.xlsx"
df.to_excel(excel_path, index=False)


In [9]:
import pandas as pd

# Load the Excel file
df = pd.read_excel("/Users/donglinhan/LargeFiles/Humidity_seeking/CYC-full.xlsx")  # Replace with your actual filename

# Limit to the first 72 hours (4320 rows)
df = df.iloc[:4320]

# Group every 60 rows (1 hour) and sum
hourly_sums = df.groupby(df.index // 60).sum()

# Adjust index to represent hours
hourly_sums.index = [f"Hour {i+1}" for i in range(len(hourly_sums))]

# Save to new Excel file
hourly_sums.to_excel("/Users/donglinhan/LargeFiles/Humidity_seeking/CYC_sums.xlsx")